In [30]:
import pandas as pd
import os
from ast import literal_eval

### load df for all QAs

In [22]:
path_to_file_list = []
path = "temp"
for p in [p for p in os.listdir(path) if not p.startswith(".")]:
    path_to_file_inner_list = []
    for f in [f for f in os.listdir(f"{path}/{p}") if not f.startswith(".")]:
        path_to_file = f"{path}/{p}/{f}"
        # print(path_to_file)
        path_to_file_inner_list.append(path_to_file)
    path_to_file_list.append(path_to_file_inner_list)

In [23]:
# path_to_file_list [categorical, span-extraction, multiple-choice]
cat_df = pd.concat([pd.read_csv(f) for f in path_to_file_list[0]])
span_df = pd.concat([pd.read_csv(f) for f in path_to_file_list[1]])
mcq_df = pd.concat([pd.read_csv(f) for f in path_to_file_list[2]])

### clean up for categorical QA

In [38]:
cat_df["logits_answer"] = cat_df["logits_answer"].str.replace("([\[\]])", "", regex=True)
cat_df["logits_answer"] = cat_df["logits_answer"].str.split()
cat_df["logits_answer_0"] = [logit[0] for logit in cat_df["logits_answer"]]
cat_df["logits_answer_1"] = [logit[1] for logit in cat_df["logits_answer"]]

In [39]:
cat_df.head()

,skill,reader,adapter,modelname,timestamp,answer,logits_answer,data_id,dataset,question,context,answer_dataset,logits_answer_0,logits_answer_1
0,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:08.066664,False,"[0.9633585, -0.6806862]",0,boolq,does ethanol take more energy make that produces,All biomass goes through at least some of thes...,False,0.9633585,-0.6806862
1,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:09.165009,True,"[0.11953807, 0.18089743]",1,boolq,is house tax and property tax are same,Property tax or 'house tax' is a local tax on ...,True,0.11953807,0.18089743
2,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:09.549410,True,"[-0.03293786, -0.00041977]",2,boolq,is pain experienced in a missing body part or ...,Phantom pain sensations are described as perce...,True,-0.03293786,-0.00041977
3,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:10.110081,True,"[0.07552637, 0.13251488]",3,boolq,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts is ...,True,0.07552637,0.13251488
4,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:10.657743,False,"[0.9214445, -0.96274555]",4,boolq,is there a difference between hydroxyzine hcl ...,Hydroxyzine preparations require a doctor's pr...,True,0.9214445,-0.96274555


In [6]:
## TODO group 

In [65]:
df = pd.DataFrame(columns=[
    "skill", "reader", "adapter", "timestamp", 
    "answer_base", "logits_answer_base",
    "answer_quantized_model", "logits_answer_quantized_model", 
    "answer_onnx_model", "logits_answer_onnx_model",
    "answer_onnx_opt_model", "logits_answer_onnx_opt_model",
    "answer_quant_onnx_model", "logits_answer_quant_onnx_model",
    "answer_quant_onnx_opt_model", "logits_answer_quant_onnx_opt_model",
    "data_id", "dataset", "question", "context", "answer_dataset"
])

In [66]:
cat_df.keys()

Index(['skill', 'reader', 'adapter', 'modelname', 'timestamp', 'answer',
       'logits_answer', 'data_id', 'dataset', 'question', 'context',
       'answer_dataset', 'logits_answer_0', 'logits_answer_1'],
      dtype='object')

In [67]:
skill = cat_df["skill"].iloc[0]
skill

'categorical'

In [68]:
skill = cat_df["skill"].iloc[0]
for reader in cat_df["reader"].unique():
    # print(reader)
    reader_cat_df = cat_df[cat_df["reader"] == reader]
    # print(len(reader_cat_df["data_id"]))

    for data_id in reader_cat_df["data_id"].unique():
        s = reader_cat_df[reader_cat_df["data_id"] == data_id].reset_index()

        adapter = s["adapter"].iloc[0]
        data_set_name = adapter
        question = s["question"].iloc[0]
        context = s["context"].iloc[0]
        answer_dataset = s["answer_dataset"].iloc[0]

        interim = {}
        for model in s["modelname"].unique():
            # print(model)
            interim[model] = (
                s[s["modelname"] == model]["answer"].values[0],
                [
                    s[s["modelname"] == model]["logits_answer_0"].values[0],
                    s[s["modelname"] == model]["logits_answer_1"].values[0],
                ]
            )
            
        # print(interim)

        df.loc[len(df)] = [
            skill, reader, adapter,
            pd.Timestamp.now(),

            interim["base"][0], [logit for logit in interim["base"][1]],
            interim["quant_base"][0], [logit for logit in interim["quant_base"][1]],
            interim["onnx"][0], [logit for logit in interim["onnx"][1]],
            interim["onnx_opt"][0], [logit for logit in interim["onnx_opt"][1]],
            interim["quant_onnx"][0], [logit for logit in interim["quant_onnx"][1]],
            interim["quant_onnx_opt"][0], [logit for logit in interim["quant_onnx_opt"][1]],
                                                                                                                            
            data_id, data_set_name, question, context, answer_dataset

        ]

        # break

        

In [69]:
df

,skill,reader,adapter,timestamp,answer_base,logits_answer_base,answer_quantized_model,logits_answer_quantized_model,answer_onnx_model,logits_answer_onnx_model,...,logits_answer_onnx_opt_model,answer_quant_onnx_model,logits_answer_quant_onnx_model,answer_quant_onnx_opt_model,logits_answer_quant_onnx_opt_model,data_id,dataset,question,context,answer_dataset
0,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.813455,True,"[0.03632838, 0.1559807]",True,"[0.04307159, 0.14856294]",False,"[1.0141207, -0.70817924]",...,"[1.0141207, -0.70817924]",False,"[0.9633585, -0.6806862]",False,"[0.9633585, -0.6806862]",0,boolq,does ethanol take more energy make that produces,All biomass goes through at least some of thes...,False
1,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.821730,True,"[0.00471915, 0.4291983]",True,"[-0.00732881, 0.44615737]",True,"[-0.40855747, 0.8667419]",...,"[-0.40855747, 0.8667419]",True,"[0.11953807, 0.18089743]",True,"[0.11953807, 0.18089743]",1,boolq,is house tax and property tax are same,Property tax or 'house tax' is a local tax on ...,True
2,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.832132,True,"[-1.280071, 1.1713798]",True,"[-1.251884, 1.1318096]",True,"[-0.8122971, 0.83803236]",...,"[-0.8122971, 0.83803236]",True,"[-0.03293786, -0.00041977]",True,"[-0.03293786, -0.00041977]",2,boolq,is pain experienced in a missing body part or ...,Phantom pain sensations are described as perce...,True
3,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.843168,True,"[-1.1285974, 1.2845722]",True,"[-1.1000056, 1.2797506]",True,"[-0.94695413, 1.1307392]",...,"[-0.94695413, 1.1307392]",True,"[0.07552637, 0.13251488]",True,"[0.07552637, 0.13251488]",3,boolq,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts is ...,True
4,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.853807,False,"[1.1531763, -1.1785511]",False,"[1.2003236, -1.2420458]",False,"[1.3443897, -1.2073131]",...,"[1.3443897, -1.2073131]",False,"[0.9214445, -0.96274555]",False,"[0.9214445, -0.96274555]",4,boolq,is there a difference between hydroxyzine hcl ...,Hydroxyzine preparations require a doctor's pr...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,categorical,roberta-base,boolq,2023-01-29 22:43:14.480709,False,"[1.1614211, -1.1667186]",False,"[1.1709174, -1.1761547]",False,"[0.7405262, -0.75474274]",...,"[0.7405262, -0.75474274]",False,"[0.9792921, -0.9989573]",False,"[0.9792921, -0.9989573]",3265,boolq,is manic depression the same as bi polar,"Bipolar disorder, previously known as manic de...",True
6536,categorical,roberta-base,boolq,2023-01-29 22:43:14.492041,True,"[-2.1512992, 2.1035373]",True,"[-2.1589963, 2.1239989]",True,"[-1.7640444, 1.6682422]",...,"[-1.7640444, 1.6682422]",True,"[-1.5305727, 1.4393224]",True,"[-1.5305727, 1.4393224]",3266,boolq,was whiskey galore based on a true story,SS Politician was an 8000-ton cargo ship owned...,True
6537,categorical,roberta-base,boolq,2023-01-29 22:43:14.502872,True,"[-3.211123, 3.1748679]",True,"[-3.2806938, 3.2134044]",True,"[-2.742387, 2.6857572]",...,"[-2.742387, 2.6857572]",True,"[-2.6072588, 2.5575714]",True,"[-2.6072588, 2.5575714]",3267,boolq,are there plants on the international space st...,Plant research continued on the International ...,True
6538,categorical,roberta-base,boolq,2023-01-29 22:43:14.513952,True,"[-0.72820544, 0.7388733]",True,"[-0.7233548, 0.7610312]",False,"[0.39946824, -0.364812]",...,"[0.39946824, -0.364812]",False,"[0.13299496, -0.08860482]",False,"[0.13299496, -0.08860482]",3268,boolq,does the hockey puck have to cross the line to...,"In ice hockey, a goal is scored when the puck ...",True


In [ ]:
# df = pd.read_csv("in.csv",converters={"Col3": lambda x: x.strip("[]").split(", ")})

In [ ]:
df.to_csv("eval_files/categorical.csv")

### clean up for span-extraction

In [43]:
span_df.tail()

,skill,reader,adapter,modelname,timestamp,answer,data_id,dataset,question,context,answer_dataset
11868,span-extraction,roberta-base,squad_v2,quant_onnx_opt,2023-01-25 13:50:57.934658,<s>,5737aafd1c456719005744ff,squad_v2,What is the seldom used force unit equal to on...,"The pound-force has a metric counterpart, less...","{'text': ['sthène', 'sthène', 'sthène', 'sthèn..."
11869,span-extraction,roberta-base,squad_v2,quant_onnx_opt,2023-01-25 13:50:58.404992,<s>,5ad28ad0d7d075001a4299cc,squad_v2,What does not have a metric counterpart?,"The pound-force has a metric counterpart, less...","{'text': [], 'answer_start': []}"
11870,span-extraction,roberta-base,squad_v2,quant_onnx_opt,2023-01-25 13:50:58.896291,<s>,5ad28ad0d7d075001a4299cd,squad_v2,What is the force exerted by standard gravity ...,"The pound-force has a metric counterpart, less...","{'text': [], 'answer_start': []}"
11871,span-extraction,roberta-base,squad_v2,quant_onnx_opt,2023-01-25 13:50:59.368777,<s>,5ad28ad0d7d075001a4299ce,squad_v2,What force leads to a commonly used unit of mass?,"The pound-force has a metric counterpart, less...","{'text': [], 'answer_start': []}"
11872,span-extraction,roberta-base,squad_v2,quant_onnx_opt,2023-01-25 13:50:59.833051,<s>,5ad28ad0d7d075001a4299cf,squad_v2,What force is part of the modern SI system?,"The pound-force has a metric counterpart, less...","{'text': [], 'answer_start': []}"
